<a href="https://colab.research.google.com/github/ricmatuk/seminariogit/blob/master/Scrap_byma_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***BYMA DATA***

## Inicializar


In [ ]:
import requests
import json
import urllib3
urllib3.disable_warnings()
import pandas as pd

## Genero una sesion para mantener las cookies.

s = requests.session()
s.get('https://open.bymadata.com.ar/#/dashboard', verify=False)

## Fuerzo los headers que necesito (no se si es indispensable, pero asi me funciono)
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
    'Origin': 'https://open.bymadata.com.ar',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://open.bymadata.com.ar/',
    'Accept-Language': 'es-US,es-419;q=0.9,es;q=0.8,en;q=0.7',
}


# Consultas generales

## Consulta de diccionario de abreviaturas y etc

In [ ]:
response = s.get('https://open.bymadata.com.ar/assets/api/langs/es.json', headers=headers)

diccionario = json.loads(response.text)

## Consulta de Market Time [?]

In [ ]:
data = '{}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/market-time', headers=headers, data=data)

response.text

'{"isWorkingDay":true,"marketClosingTime":"17:00:00","timezone":"GMT-03:00","marketOpeningTime":"11:00:00"}'

## Consulta de indices renta variable

El resto de las consultas del dashboard no las puedo chequear porque los capos devuelven todo vacio a esta hora. Mañana intento completar el resto. 


In [ ]:
data = '{"Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/index-price', headers=headers, data=data, verify=False)
indices = json.loads(response.text)['data']
df_indices = pd.DataFrame(indices)

df_indices

,symbol,sourceCode,country,isRate,minValue,highValue,price,description,closingPrice,previousClosingPrice,variation
0,G,BYMA,Argentina,False,3403501.55,3434387.03,3416055.45,S&P BYMA Indice General,0,3422650.99,-0.001927
1,M,BYMA,Argentina,False,81564.33,82293.38,81784.58,S&P MERVAL,0,81947.65,-0.001990
2,SPBYCAP,BYMA,Argentina,False,2703.92,2809.85,2758.18,S&P BYMA Construccion,0,2809.85,-0.018389
3,SPBYMAIG10,BYMA,Argentina,False,1687.91,1704.48,1687.91,S&P BYMA Energia,0,1696.36,-0.004981
4,SPBYMAIG15,BYMA,Argentina,False,5308.31,5399.79,5352.41,S&P BYMA Bs Intermedios,0,5382.10,-0.005516
5,SPBYMAIG20,BYMA,Argentina,False,2283.77,2300.29,2285.27,S&P BYMA Ind y Bs de Cap,0,2300.29,-0.006530
6,SPBYMAIG25,BYMA,Argentina,False,9878.57,9918.44,9878.57,S&P BYMA Cons No Esencial,0,9909.00,-0.003071
7,SPBYMAIG30,BYMA,Argentina,False,3460.36,3473.90,3473.90,S&P BYMA Consumo Masivo,0,3460.36,0.003913
8,SPBYMAIG40,BYMA,Argentina,False,1720.32,1739.82,1731.03,S&P BYMA Serv Financieros,0,1731.14,-0.000064
9,SPBYMAIG50,BYMA,Argentina,False,2400.35,2447.49,2447.49,S&P BYMA Telecomunicacion,0,2417.30,0.012489


## Consulta de ultima cotizacion

In [ ]:
data = '{"symbol":"ALUA","settlementType":"1","Content-Type":"application/json"}'  ## Cambiar symbol y/o settlementType para diferentes especies y plazos (3 = 48, 2 = 24, 1 = CI)
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/fichatecnica/especies/cotizacion', headers=headers, data=data)
last = json.loads(response.text)

last['data']

[{'bidPrice': 99.3,
  'closingPrice': 0,
  'denominationCcy': 'ARS',
  'imbalance': -0.0104,
  'market': 'BYMA',
  'numberOfOrders': 27,
  'offerPrice': 99.6,
  'openingPrice': 101,
  'previousClosingPrice': 100.75,
  'previousSettlementPrice': 100.75,
  'quantityBid': 33,
  'quantityOffer': 100,
  'securityDesc': '',
  'securitySubType': 'M',
  'securityType': 'CS',
  'settlementPrice': 99.7,
  'settlementType': '1',
  'symbol': 'ALUA',
  'tickDirection': 0,
  'trade': 99.7,
  'tradeHour': '14:28:57',
  'tradeVolume': 3690,
  'tradingHighPrice': 101,
  'tradingLowPrice': 98.6,
  'volume': 3690,
  'volumeAmount': 366039,
  'vwap': 99.197561}]

## Consulta de cierres diarios 



In [ ]:
params = (
    ('symbol', 'AL30 CDO'),  ## Nombre de especie y plazo (CDO / 24HS / 48HS)
    ('resolution', 'D'), ## puede ser D - Diario, S - Semanal, M - Mensual, 
    ('from', '1605576698'), ## Unix epoch, se deberia poder armar como int(datetime.datetime(AÑO,MES,DIA,HORA,MINUTOS).timestamp())
    ('to', '1639445558'),
)

response = s.get('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/chart/historical-series/history', params=params)

"""
trae un diccionario con 7 keys, 
 c = close
 h = high
 l = low
 o = open
 t = timestamp
 s = status
 v = volumen
"""

df_historico = pd.DataFrame(json.loads(response.text))  # lo convierto en df
df_historico['t'] = pd.to_datetime(df_historico['t'], unit='s').dt.normalize() # cambio el formato de la fecha. 

df_historico.rename({'s':'status', 't':'timestamp', 'c':'close', 'o':'open', 'h':'high', 'l':'low', 'v':'volume'}, axis = 1, inplace=True)

df_historico
# supongo que se deberia dropear la columna status, pero bue...

,status,timestamp,close,open,high,low,volume
0,ok,2020-11-16,5769.000,5705.000,5796.50,5665.000,49819958
1,ok,2020-11-16,5758.190,5681.450,5900.00,39.920,65074417
2,ok,2020-11-17,5807.000,5770.000,5848.00,5767.000,52773366
3,ok,2020-11-17,5808.858,5760.370,5913.92,5632.000,78251686
4,ok,2020-11-18,5844.500,5830.000,5849.00,5808.000,55173433
...,...,...,...,...,...,...,...
515,ok,2021-12-09,6637.026,6770.000,6820.00,6549.732,99364775
516,ok,2021-12-10,6577.500,6675.500,6675.50,6549.000,14918508
517,ok,2021-12-10,6700.000,6777.273,6819.50,6462.750,120505088
518,ok,2021-12-13,6457.000,6675.000,6675.00,6405.000,19151730


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1, subplot_titles=('Candlestick', 'Volume'),
                    row_width=[0.3, 0.7])

fig.add_trace(go.Candlestick(x=df_historico['timestamp'], 
                             open=df_historico['open'], 
                             high=df_historico['high'], 
                             low=df_historico['low'], 
                             close=df_historico['close']),
                             row=1, col=1)

fig.add_trace(go.Bar(x=df_historico['timestamp'], y=df_historico['volume'], showlegend=False), row=2, col=1)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

## Consulta de totales negociados

In [ ]:
data = '{"Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/total-negotiated', headers=headers, data=data)
totales_negociados = json.loads(response.text)
df_totales = pd.DataFrame(totales_negociados)

# Para hacerlo mas legible, reemplazo abreviaturas por el valor del diccionario descargado al principio
df_totales['symbol'] = df_totales['symbol'].replace(diccionario, regex=True)
df_totales['assetType'] = df_totales['assetType'].replace(diccionario, regex=True)
df_totales['parentKey'] = df_totales['parentKey'].replace(diccionario, regex=True)

df_totales

,settlementType,symbol,amount,level,assetType,order,parentKey
0,1,Total Negociado BYMA,4235286806,0,Total Negociado BYMA,1,NaN
1,1,Renta Variable,112112018,0,Renta Variable,2,NaN
2,1,Acciones,17046313,1,Acciones,3,Renta Variable
3,1,Cedears,95065705,1,Cedears,4,Renta Variable
4,1,Ejercicios,0,1,Ejercicios,5,Renta Variable
5,1,Renta Fija,1430601130,0,Renta Fija,6,NaN
6,1,Renta FijaPPT,428197986,1,Renta FijaPPT,7,Renta Fija
7,1,- Títulos Públicos,402460731,2,- Títulos Públicos,8,Renta FijaPPT
8,1,- Obligaciones Negociables,25737255,2,- Obligaciones Negociables,9,Renta FijaPPT
9,1,Renta FijaSEN,1002403144,1,Renta FijaSEN,10,Renta Fija


## Consulta bonos iamc

In [ ]:
data = '{"page_number":1, "page_size":500, "Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/seriesHistoricas/iamc/bonos', headers=headers, data=data)
bonos_iamc = json.loads(response.text)
df_bonos_iamc = pd.DataFrame(bonos_iamc['data'])

df_bonos_iamc

,descripcion,symbol,hora,notas,paridad,fechaCot,dm,tirAnual,cotizacion,vTecnico,vr,intCorr,rentaAnual,isin
0,BONO REP. ARGENTINA EN U$S REGS. 5.625% - VTO....,A2E2,15:23,,0.515754,2020-08-24 00:00:00.0,0.919003,0.760724,6100,103.359,100.0,3.35938,Fija=5.625,US040114HK99
1,BONO REP. ARGENTINA U$S REGS. 5.625% - V. 26/0...,A2E2D,15:30,,0.474074,2020-08-21 00:00:00.0,0.873835,0.882110,49,103.359,100.0,3.35938,Fija=5.625,US040114HK99
2,BONOS INTERNAC. REP. ARG. U$S 4.625% - V. 11/0...,A2E3,13:33,,0.417512,2020-08-11 00:00:00.0,1.587509,0.577606,5160,102.942,100.0,2.94201,Fija=4.625,US040114HP86
3,BONO INTERNAC. REP. ARG. U$S 4.625% - V. 11/01...,A2E3D,13:22,,0.286569,2020-07-08 00:00:00.0,1.334394,0.905221,41.66,102.942,100.0,2.94201,Fija=4.625,US040114HP86
4,BONO REP. ARGENTINA 6.875% EN U$D - VTO. 26/01...,A2E7,12:58,,0.420045,2020-08-28 00:00:00.0,3.536264,0.292171,5250,104.106,100.0,4.1059,Fija=6.875,US040114HL72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,O.N. YPF S.A. CLASE 16,YMCHO,16:04,,0.83667,2021-12-13 00:00:00.0,2.300776,0.152513,17299,100.367,100.0,0.366667,Fija=4,USP989MJBR17
146,O.N. YPF S.A. CLASE 17,YMCIO,17:00,,0.633503,2021-12-13 00:00:00.0,4.284005,0.184220,13200,101.146,100.0,1.14583,Fija=2.5,USP989MJBS99
147,O.N. YPF S.A. CLASE 18,YMCJO,14:59,,0.531821,2021-12-13 00:00:00.0,6.444701,0.157737,10990,100.312,100.0,0.3125,Fija=1.5,USP989MJBT72
148,O.N. YPF CLASE 28 8.75% EN U$S VTO. 04/04/2024,YPCUO,15:22,,0.878961,2021-12-13 00:00:00.0,1.119305,0.205836,18419.5,101.726,100.0,1.72569,Fija=8.75,USP989MJAY76


# Datos de las empresas

## Datos de la especie

In [ ]:
data = '{"symbol":"ALUA","Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/fichatecnica/especies/general', headers=headers, data=data)
datos_especie = json.loads(response.text)

datos_especie['data'][0] ## Ni idea porque repite el mismo dato tantas veces

{'capitalizacion': 278320000000,
 'codigoIsin': 'ARALUA010258',
 'denominacion': 'Ordinarias Escriturales (1 Voto)',
 'descripcion': 'Pesos',
 'emisor': 'ALUAR ALUMINIO ARGENTINO S.A.',
 'insType': 'EQUITY',
 'lider': 'Si',
 'montoAutorizado': 2800000000,
 'tipoEspecie': 'Acciones',
 'valorNominal': 1}

## Datos generales

In [ ]:
data = '{"symbol":"ALUA","Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/fichatecnica/sociedades/general', headers=headers, data=data)
datos_generales = json.loads(response.text)

datos_generales['data']

[{'actividad': 'Explotación de una Planta de Aluminio Primario.',
  'calle': 'Marcelo Torcuato de Alvear 590',
  'cierreEjercicio': '30/06',
  'codPostal': 'C1058AAF',
  'dpto': '',
  'email': 'asjurid@infovia.com.ar',
  'fechaConstitucion': '26 de Junio de 1970',
  'localidad': 'Capital Federal',
  'logo': 13919,
  'pais': 'Argentina',
  'piso': '3°',
  'provinciaEstado': 'Capital Federal',
  'telefono': '(011) 4313-7593 / 4311-9026',
  'website': 'www.aluar.com.ar'}]

## Datos de responsables de relaciones con el mercado

In [ ]:
data = '{"symbol":"ALUA","Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/fichatecnica/sociedades/responsables', headers=headers, data=data)
datos_responsable = json.loads(response.text)

datos_responsable['data']

[{'cargo': 'Titular',
  'cargoEmpresa': '',
  'fechaDesignacion': '2001-11-30 00:00:00.0',
  'responsableMercado': 'Martínez Costa, Alberto Eduardo'},
 {'cargo': 'Suplente',
  'cargoEmpresa': '',
  'fechaDesignacion': '2006-07-12 00:00:00.0',
  'responsableMercado': 'Falcón, Miguel Juan'}]

## Datos del management

In [ ]:
data = '{"symbol":"ALUA","Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/fichatecnica/sociedades/administracion', headers=headers, data=data)
datos_management = json.loads(response.text)
df_management = pd.DataFrame(datos_management['data'])

df_management

,cargo,persona
0,Presidente,MARTINEZ COSTA ALBERTO EDUARDO
1,Vicepresidente,FRIEDENTHAL DANIEL
2,Director Titular,KLAINER DANIEL
3,Director Titular,LEVINAS MARTIN JOSE
4,Director Titular,FRIEDENTHAL DIEGO
5,Director Titular,PUCHES MIROSLAVO
6,Director Titular,GOMEZ PRIETO MARCELO RODOLFO
7,Director Titular,FALCON MIGUEL JUAN
8,Director Titular,MARCEL ALBERTO ELEODORO
9,Director Titular,MIGUELES PEDRO GUILLERMO


## Datos del balance

In [ ]:
params = (('symbol', 'ALUA'),)
response = s.get('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bolsar/balances', headers=headers, params=params)
datos_balance = json.loads(response.text)
datos_balance['data']
#Aca armo un df, pero ni idea si todas las especies tienen la misma estructura, habria que mirarlo bien, ademas se pierde la fecha, etc. Falta laburarlo.
df_balance = pd.DataFrame(datos_balance['data'][0]['Cuentas'])

df_balance

,Nombre,Importe
0,Ingresos de actividades ordinarias,2.786378e+10
1,Costo De Ventas Y/O Servicios,-1.781838e+10
2,Ganancia (Pérdida) bruta,1.004541e+10
3,Honorarios a Directores y Síndicos,-5.168052e+06
4,Otros gastos de administracion,-5.718754e+08
5,Gastos de comercializacion y/o distribucion,-1.067912e+09
6,Otros gastos operativos,-1.889474e+07
7,Otros Ingresos,NaN
8,Depreciaciones y Amortizaciones,-1.994609e+09
9,Ganancia (pérdida) de actividades operativas,6.386949e+09


## Descarga de balances historicos 
(falta terminar de desarrollar)

In [ ]:

data = '{"symbol":"ALUA","Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/seriesHistoricas/balances', headers=headers, data=data)
balances_historicos = json.loads(response.text)

balances_historicos['data'][0]

## TOMAR EL NRO DE ARCHIVO DE ESTE LISTADO Y CONCATENARLO A ESTE URL https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/sba/download/
## POR EJEMPLO https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/sba/download/396964
## PERO NO TUVE TIEMPO DE ARMAR LA DESCARGA. 


{'archivo': 396964,
 'balancesArchivo': 'https://www.bolsar.com/Downloads.aspx?Id=396964',
 'balancesFechaEstadoContable': '2021-09-30 00:00:00.0',
 'balancesFechaPresentacion': '2021-11-11 00:00:00.0',
 'tipoArchivo': 'pdf'}

# Renta Variable


## Acciones panel Lideres

In [ ]:
data = '{"excludeZeroPxAndQty":false,"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/leading-equity', headers=headers, data=data)
panel_acciones_lideres = json.loads(response.text)
df_panel_acciones_lideres = pd.DataFrame(panel_acciones_lideres['data'])

df_panel_acciones_lideres

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,tradeHour,securityType,closingPrice,settlementPrice,quantityBid
0,2560.0,ALUA,0.0060,99.40,101.00,99.789609,31,99.40,-1,,M,99.40,3,1007,100.00,ARS,100.00,99.40,BYMA,255461.40,2560.0,100.00,11:04:27,CS,0,100.00,886.0
1,151.0,BBAR,0.0059,201.75,202.95,202.583444,11,202.50,0,,M,201.75,3,472,202.95,ARS,202.60,202.50,BYMA,30590.10,151.0,202.95,11:02:50,CS,0,202.95,24.0
2,1517.0,BMA,0.0092,280.65,283.25,283.604878,10,282.00,0,,M,280.65,3,554,283.85,ARS,281.25,281.00,BYMA,430228.60,1517.0,283.25,11:04:52,CS,0,283.25,900.0
3,14.0,BYMA,-0.0064,1010.50,1004.00,1020.428571,5,1033.00,-1,,M,1010.50,3,27,1033.00,ARS,970.00,1004.00,BYMA,14286.00,14.0,1004.00,11:04:27,CS,0,1004.00,200.0
4,1494.0,CEPU,-0.0022,67.15,67.00,67.109706,7,67.20,-1,,M,67.15,3,1453,67.20,ARS,66.10,67.00,BYMA,100261.90,1494.0,67.00,11:02:49,CS,0,67.00,45.0
5,3930.0,COME,0.0053,5.58,5.61,5.536107,18,5.53,0,,M,5.58,3,3907,5.61,ARS,5.54,5.53,BYMA,21756.90,3930.0,5.61,11:04:59,CS,0,5.61,536.0
6,50.0,CRES,0.0029,101.70,101.50,102.000000,3,102.00,-1,,M,101.70,3,505,102.00,ARS,101.20,102.00,BYMA,5100.00,50.0,102.00,11:04:55,CS,0,102.00,10.0
7,21.0,CVH,-0.0182,519.50,517.50,517.023809,7,517.50,2,,M,519.50,3,158,517.50,ARS,510.00,510.00,BYMA,10857.50,21.0,510.00,11:02:24,CS,0,510.00,63.0
8,1910.0,EDN,-0.0081,54.95,54.50,55.071335,14,55.35,-1,,M,54.95,3,1327,55.35,ARS,54.05,54.50,BYMA,105186.25,1910.0,54.50,11:04:52,CS,0,54.50,886.0
9,7967.0,GGAL,0.0005,187.90,188.00,187.593743,55,188.00,-1,,M,187.90,3,1850,188.45,ARS,187.30,185.00,BYMA,1494559.35,7967.0,188.00,11:04:31,CS,0,188.00,500.0


## Acciones panel General


In [ ]:
data = '{"excludeZeroPxAndQty":false,"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/general-equity', headers=headers, data=data)
panel_acciones_general = json.loads(response.text)
df_panel_acciones_general = pd.DataFrame(panel_acciones_general['data'])

df_panel_acciones_general

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,securityType,closingPrice,settlementPrice,quantityBid,tickDirection,tradeHour
0,0.0,ABBA,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
1,0.0,ABBAC,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
2,0.0,ABBAD,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
3,0.0,ABMT,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
4,0.0,ABMTC,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,0.0,VALOC,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
294,0.0,VALOD,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
295,0.0,YPFDB,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN
296,0.0,YPFDC,0.0,0.0,0.0,0.0,0,0.0,,G,0.0,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,CS,0.0,0.0,0.0,NaN,NaN


## CEDEARS 


In [ ]:
data = '{"excludeZeroPxAndQty":false,"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/cedears', headers=headers, data=data)
panel_cedears = json.loads(response.text)
panel_cedears
df_panel_cedears = pd.DataFrame(panel_cedears)

df_panel_cedears

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,securityType,closingPrice,settlementPrice,quantityBid,tickDirection,tradeHour
0,0.0,AABA,0.0000,0.000000,0.0,0.000000,0,0.00,,,0.000000,3,0.0,0.00,ARS,0.0,0.00,BYMA,0.0,0.0,0.00,CD,0.000000,0.00,0.0,NaN,NaN
1,942.0,AAPL,-0.0215,3644.500000,3579.0,3573.583864,98,3599.50,,,3644.500000,3,2.0,3599.50,ARS,3563.5,3561.00,BYMA,3366316.0,942.0,3566.00,CD,0.000000,3566.00,1617.0,2.0,11:04:45
2,0.0,AAPLB,0.0000,0.000000,0.0,0.000000,0,0.00,,,0.000000,3,0.0,0.00,ARS,0.0,0.00,BYMA,0.0,0.0,0.00,CD,0.000000,0.00,0.0,NaN,NaN
3,0.0,AAPLC,0.0000,17.900000,0.0,0.000000,0,0.00,,,17.900000,3,0.0,0.00,EXT,0.0,0.00,BYMA,0.0,0.0,0.00,CD,0.000000,0.00,0.0,-1.0,NaN
4,10.0,AAPLD,-0.0265,18.850000,18.8,18.550000,2,18.75,,,18.850000,3,5.0,18.75,USD,18.0,18.35,BYMA,185.5,10.0,18.35,CD,0.000000,18.35,30.0,2.0,11:04:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0.0,YZCA,0.0000,421.012481,0.0,0.000000,0,0.00,,,421.012481,3,0.0,0.00,ARS,0.0,0.00,BYMA,0.0,0.0,0.00,CD,421.012481,0.00,0.0,-1.0,NaN
743,0.0,YZCAC,0.0000,0.509896,0.0,0.000000,0,0.00,,,0.509896,3,0.0,0.00,EXT,0.0,0.00,BYMA,0.0,0.0,0.00,CD,0.509896,0.00,0.0,-1.0,NaN
744,60.0,ZM,-0.0068,804.000000,806.0,798.983333,3,799.00,,,804.000000,3,611.0,799.00,ARS,790.0,798.50,BYMA,47939.0,60.0,798.50,CD,0.000000,798.50,1.0,2.0,11:01:16
745,0.0,ZMC,0.0000,6.150000,0.0,0.000000,0,0.00,,,6.150000,3,0.0,0.00,EXT,0.0,0.00,BYMA,0.0,0.0,0.00,CD,6.150000,0.00,0.0,-1.0,NaN


# Renta Fija

## Titulos Publicos


In [ ]:
data = '{"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/public-bonds', headers=headers, data=data)
panel_bonos = json.loads(response.text)
df_panel_bonos = pd.DataFrame(panel_bonos['data'])

df_panel_bonos

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,tradeHour,securityType,closingPrice,settlementPrice,quantityBid
0,3462,AA22,0.0000,107.00,107.00,106.958218,8,106.95,-1,,B,2022-04-03,107.00,3,3446,107.0,ARS,99.05,106.95,BYMA,3702.90,3462,107.0,110,11:02:14,GO,0,107.0,60000
1,0,AE38,0.0000,7195.00,7250.00,0.000000,0,0.00,-1,,B,2038-07-09,7195.00,3,6879,0.0,ARS,6901.00,0.00,BYMA,0.00,0,0.0,6051,NaN,GO,0,0.0,282
2,0,AE38C,0.0000,37.60,0.00,0.000000,0,0.00,-1,,B,2038-07-09,37.60,3,0,0.0,EXT,33.00,0.00,BYMA,0.00,0,0.0,6051,NaN,GO,0,0.0,50000
3,280,AE38D,0.0013,37.05,38.75,37.100000,1,37.10,0,,B,2038-07-09,37.05,3,16673,37.1,USD,36.00,37.10,BYMA,103.88,280,37.1,6051,11:00:04,GO,0,37.1,34882
4,75,AL29,0.0102,6909.50,6979.50,6980.000000,2,6980.00,-1,,B,2029-07-09,6909.50,3,1000,6980.0,ARS,6860.00,6980.00,BYMA,5235.00,75,6980.0,2764,11:00,GO,0,6980.0,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,10549,TX22,0.0066,187.75,188.95,188.993269,4,188.00,0,,B,2022-03-18,187.75,3,2154,189.0,ARS,188.00,188.00,BYMA,19936.90,10549,189.0,94,11:02:27,GO,0,189.0,9999
63,1188,TX23,0.0108,185.00,187.00,187.000000,2,187.00,-1,,B,2023-03-25,185.00,3,4998812,187.0,ARS,185.00,187.00,BYMA,2221.56,1188,187.0,466,11:02:52,GO,0,187.0,8108
64,0,TX24,0.0000,181.50,183.95,0.000000,0,0.00,-1,,B,2024-03-25,181.50,3,39728,0.0,ARS,180.20,0.00,BYMA,0.00,0,0.0,832,NaN,GO,0,0.0,99
65,1234,TX26,0.0000,148.00,150.00,148.000000,2,148.00,-1,,B,2026-11-09,148.00,3,217926,148.0,ARS,148.00,148.00,BYMA,1826.32,1234,148.0,1791,11:02:36,GO,0,148.0,64091


## Letras


In [ ]:
data = '{"excludeZeroPxAndQty":false,"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/lebacs', headers=headers, data=data)
panel_letras = json.loads(response.text)
df_panel_letras = pd.DataFrame(panel_letras['data'])

df_panel_letras

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid,tradeHour
0,0.0,BAM22,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-03-25,0.000,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,101,GO,0.0,0.0,0.0,NaN
1,0.0,BAM2C,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-03-25,0.000,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,101,GO,0.0,0.0,0.0,NaN
2,0.0,BAM2D,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-03-25,0.000,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,101,GO,0.0,0.0,0.0,NaN
3,0.0,BBA22,0.0,102.000,102.0,0.0,0,0.0,-1.0,,E,2022-04-29,102.000,3,30000.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,136,GO,0.0,0.0,0.0,NaN
4,0.0,BBD1C,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2021-12-17,0.000,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,3,GO,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,0.0,XY2C,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-05-23,0.000,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,160,GO,0.0,0.0,0.0,NaN
228,0.0,XY2D,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-05-23,0.000,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,160,GO,0.0,0.0,0.0,NaN
229,0.0,XY2X,0.0,125.575,0.0,0.0,0,0.0,-1.0,,E,2022-05-23,125.575,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,160,GO,0.0,0.0,0.0,NaN
230,0.0,XY2Y,0.0,0.000,0.0,0.0,0,0.0,-1.0,,E,2022-05-23,0.000,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,160,GO,0.0,0.0,0.0,NaN


## Obligaciones Negociables


In [ ]:
data = '{"excludeZeroPxAndQty":false,"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/negociable-obligations', headers=headers, data=data)
panel_ons = json.loads(response.text)
df_panel_ons = pd.DataFrame(panel_ons)

df_panel_ons

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid,tradeHour
0,0.0,AECAC,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2024-02-02,0.00,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,780,CORP,0.0,0.0,0.0,NaN
1,0.0,AECAD,0.0,74.76,0.0,0.0,0,0.0,-1.0,,,2024-02-02,74.76,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,780,CORP,0.0,0.0,0.0,NaN
2,0.0,AECAO,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2024-02-02,0.00,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,780,CORP,0.0,0.0,0.0,NaN
3,0.0,AECAX,0.0,14088.47,0.0,0.0,0,0.0,-1.0,,,2024-02-02,14088.47,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,780,CORP,0.0,0.0,0.0,NaN
4,0.0,AECAY,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2024-02-02,0.00,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,780,CORP,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,0.0,ZA1AD,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2022-02-24,0.00,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,72,CORP,0.0,0.0,0.0,NaN
1661,0.0,ZA1AP,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2022-02-24,0.00,3,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,72,CORP,0.0,0.0,0.0,NaN
1662,0.0,ZA1BC,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2024-08-24,0.00,3,0.0,0.0,EXT,0.0,0.0,BYMA,0.0,0.0,0.0,984,CORP,0.0,0.0,0.0,NaN
1663,0.0,ZA1BD,0.0,0.00,0.0,0.0,0,0.0,-1.0,,,2024-08-24,0.00,3,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,984,CORP,0.0,0.0,0.0,NaN


## Cauciones


In [ ]:
data = '{"excludeZeroPxAndQty":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad, determina plazo de listado
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/cauciones', headers=headers, data=data)
panel_cauciones = json.loads(response.text)
df_panel_cauciones = pd.DataFrame(panel_cauciones)

df_panel_cauciones

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,underlyingSymbol,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid,tradeHour
0,0.0,DOLAR-0102-U-CT-USD,0.0,0.000,0.0,0.0,0,0.0,-1.0,DOLAR,,,2022-02-01,0.000,1,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,49,QS,0.0,0.0,0.0,NaN
1,0.0,DOLAR-0103-U-CT-USD,0.0,0.000,0.0,0.0,0,0.0,-1.0,DOLAR,,,2022-03-01,0.000,1,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,77,QS,0.0,0.0,0.0,NaN
2,0.0,DOLAR-0104-U-CT-USD,0.0,0.000,0.0,0.0,0,0.0,-1.0,DOLAR,,,2022-04-01,0.000,1,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,108,QS,0.0,0.0,0.0,NaN
3,0.0,DOLAR-0202-U-CT-USD,0.0,0.000,0.0,0.0,0,0.0,-1.0,DOLAR,,,2022-02-02,0.000,1,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,50,QS,0.0,0.0,0.0,NaN
4,0.0,DOLAR-0203-U-CT-USD,0.0,0.027,0.0,0.0,0,0.0,-1.0,DOLAR,,,2022-03-02,0.027,1,0.0,0.0,USD,0.0,0.0,BYMA,0.0,0.0,0.0,78,QS,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.0,PESOS-3012-U-CT-ARS,0.0,0.340,0.0,0.0,0,0.0,-1.0,PESOS,,,2021-12-30,0.340,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,QS,0.0,0.0,0.0,NaN
184,0.0,PESOS-3101-U-CT-ARS,0.0,0.280,0.0,0.0,0,0.0,-1.0,PESOS,,,2022-01-31,0.280,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,48,QS,0.0,0.0,0.0,NaN
185,0.0,PESOS-3103-U-CT-ARS,0.0,0.000,0.0,0.0,0,0.0,-1.0,PESOS,,,2022-03-31,0.000,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,107,QS,0.0,0.0,0.0,NaN
186,0.0,PESOS-3108-U-CT-ARS,0.0,0.000,0.0,0.0,0,0.0,NaN,NaN,,,2022-08-31,0.000,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,260,QS,0.0,0.0,0.0,NaN


## Plazos por lote


In [ ]:
data = '{"Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/plazo-lote', headers=headers, data=data)
panel_plazos_lote = json.loads(response.text)
df_panel_plazos_lote = pd.DataFrame(panel_plazos_lote['data'])

df_panel_plazos_lote

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,underlyingSymbol,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid,tickDirection
0,0.0,A2E2-2602-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,A2E2,,P,2022-02-26,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,74,T,0.0,0.0,0.0,NaN
1,0.0,AA22-2502-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,P,2022-02-25,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,73,T,0.0,0.0,0.0,-1.0
2,0.0,AA22-3012-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,P,2021-12-30,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,T,0.0,0.0,0.0,-1.0
3,0.0,AA22-3101-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,P,2022-01-31,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,48,T,0.0,0.0,0.0,-1.0
4,0.0,AA22-3103-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,P,2022-03-31,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,107,T,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,0.0,YPFD-2904-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,P,2022-04-29,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,136,T,0.0,0.0,0.0,-1.0
991,0.0,YPFD-3012-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,P,2021-12-30,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,T,0.0,0.0,0.0,-1.0
992,0.0,YPFD-3101-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,P,2022-01-31,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,48,T,0.0,0.0,0.0,-1.0
993,0.0,YPFD-3103-T-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,P,2022-03-31,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,107,T,0.0,0.0,0.0,-1.0


## Prestamos Liquidacion


In [ ]:
data = '{"excludeZeroPxAndQty":false,"Content-Type":"application/json"}' ## excluir especies sin precio y cantidad
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/loans-liquidation', headers=headers, data=data)
panel_prestamos = json.loads(response.text)
df_panel_prestamos = pd.DataFrame(panel_prestamos['data'])

df_panel_prestamos

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,underlyingSymbol,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid,tickDirection
0,0.0,AA22-1312-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,PQ,2021-12-13,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,-1,TERM,0.0,0.0,0.0,NaN
1,0.0,AA22-1412-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AA22,,PQ,2021-12-14,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,0,TERM,0.0,0.0,0.0,NaN
2,0.0,AAPL-1312-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AAPL,,PQ,2021-12-13,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,-1,TERM,0.0,0.0,0.0,NaN
3,0.0,AAPL-1412-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AAPL,,PQ,2021-12-14,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,0,TERM,0.0,0.0,0.0,NaN
4,0.0,AAPL-1512-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,AAPL,,PQ,2021-12-15,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,1,TERM,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,0.0,YPFD-1412-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,PQ,2021-12-14,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,0,TERM,0.0,0.0,0.0,NaN
725,0.0,YPFD-1512-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,YPFD,,PQ,2021-12-15,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,1,TERM,0.0,0.0,0.0,-1.0
726,0.0,ZM-1312-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,ZM,,PQ,2021-12-13,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,-1,TERM,0.0,0.0,0.0,NaN
727,0.0,ZM-1412-L-CT-ARS,0.0,0.0,0.0,0.0,0,0.0,ZM,,PQ,2021-12-14,0.0,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,0,TERM,0.0,0.0,0.0,NaN


## Prestamos venta en corto

In [ ]:
data = '{"excludeZeroPxAndQty":false,"Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/loans-short-sale', headers=headers, data=data)
panel_prestamo_corto = json.loads(response.text)
df_panel_prestamo_corto = pd.DataFrame(panel_prestamo_corto['data'])

df_panel_prestamo_corto

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,underlyingSymbol,securityDesc,securitySubType,maturityDate,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,denominationCcy,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,daysToMaturity,securityType,closingPrice,settlementPrice,quantityBid
0,0.0,AL30-3012-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,AL30,,VQ,2021-12-30,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,STN,0.0,0.0,0.0
1,0.0,AL41-3012-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,AL41,,VQ,2021-12-30,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,STN,0.0,0.0,0.0
2,0.0,BMA-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,BMA,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
3,0.0,BYMA-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,BYMA,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
4,0.0,CRES-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,CRES,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
5,0.0,CVH-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,CVH,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
6,0.0,GD30-3012-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,GD30,,VQ,2021-12-30,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,16,STN,0.0,0.0,0.0
7,0.0,GGAL-2112-V-CT-ARS,0.0,0.03,0.06,0.0,0,0.0,-1.0,GGAL,,VQ,2021-12-21,0.03,1,53585.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
8,0.0,LOMA-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,LOMA,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0
9,0.0,MELI-2112-V-CT-ARS,0.0,0.00,0.00,0.0,0,0.0,-1.0,MELI,,VQ,2021-12-21,0.00,1,0.0,0.0,ARS,0.0,0.0,BYMA,0.0,0.0,0.0,7,STN,0.0,0.0,0.0



## Fideicomisos

In [ ]:
data = '{"page_number":1,"page_size": 5000, "Content-Type":"application/json"}'
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/financial-trusts', headers=headers, data=data)
fideicomisos = json.loads(response.text)
df_fideicomisos = pd.DataFrame(fideicomisos['data'])

df_fideicomisos

,simbolo,closePriceHom,avgPrice,openPrice,varPrice,openPriceHom,varPriceHom,fecha,minPriceHom,minPrice,maxPriceHom,closePrice,maxPrice,cantOper
0,BT43B,97.618,97.618,97.618,0.000,97.618,0.000,2017-06-05 00:00:00.0,97.618,97.618,97.618,97.618,97.618,1
1,BT43C,96.125,96.125,96.125,0.000,96.125,0.000,2017-06-02 00:00:00.0,96.125,96.125,96.125,96.125,96.125,1
2,BT46A,103.380,103.380,103.38,0.000,103.38,0.000,2018-02-28 00:00:00.0,103.380,103.380,103.380,103.380,103.380,1
3,BT46B,97.250,97.250,97.25,0.000,97.25,0.000,2019-02-21 00:00:00.0,97.250,97.250,97.250,97.250,97.250,1
4,BT47C,124.050,124.050,124.05,0.000,124.05,0.000,2019-04-17 00:00:00.0,124.050,124.050,124.050,124.050,124.050,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,TV16A,68.350,67.000,68.35,0.515,68.35,0.515,2014-12-15 00:00:00.0,68.350,68.350,68.350,68.350,68.350,2
164,T102A,17.000,17.050,17.1,0.000,17.1,0.000,2018-09-19 00:00:00.0,17.000,17.000,17.100,17.000,17.100,2
165,VC07C,4240.000,4240.000,4240,3.415,4240,3.302,2019-02-15 00:00:00.0,4240.000,4240.000,4240.000,4240.000,4240.000,1
166,VC08A,7225.000,7225.000,7225,1.134,7225,1.121,2019-12-04 00:00:00.0,7225.000,7225.000,7225.000,7225.000,7225.000,2


## SENEBI - Bonos

In [ ]:
data = '{"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}'  ## Plazos de negociacion para listar. 
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/senebi-bonds', headers=headers,  data=data)
senebi_bonos = json.loads(response.text)
df_senebi_bonos = pd.DataFrame(senebi_bonos['data'])

df_senebi_bonos

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,tradeHour,securityType,closingPrice,settlementPrice,quantityBid
0,120000000,AA22,0.0028,105.950,0,106.1582,0,105.990,-1,,B,105.950,3,0,106.250,0,105.990,SENEBI,127266000,120000000,106.250,14:00:08,SEN,0,0,0
1,566380,AE38,-0.0199,7320.000,0,7174.1014,0,7175.000,-1,,B,7320.000,3,0,7175.000,0,7174.000,SENEBI,40632765,566380,7174.000,14:00:09,SEN,0,0,0
2,4469000,AL30,-0.0046,6580.000,0,6526.6957,0,6484.770,-1,,B,6580.000,3,0,6550.000,0,6484.770,SENEBI,290730637,4469000,6550.000,14:00:09,SEN,0,0,0
3,3616665,AL35,-0.0581,6625.000,0,6265.7691,0,6262.500,2,,B,6625.000,3,0,6284.500,0,6225.000,SENEBI,226667045,3616665,6240.000,14:00:09,SEN,0,0,0
4,841856,BA37D,-0.0084,9238.000,0,9144.1348,0,9067.360,-1,,B,9238.000,3,0,9160.000,0,9067.360,SENEBI,76940924,841856,9160.000,14:00:11,SEN,0,0,0
5,211688,BA7DC,0.0752,40.550,0,43.6000,0,43.600,-1,,B,40.550,3,0,43.600,0,43.600,SENEBI,92295,211688,43.600,14:00:11,SEN,0,0,0
6,500000000,BDC24,-0.0005,97.800,0,97.6726,0,97.550,-1,,B,97.800,3,0,97.750,0,97.550,SENEBI,487950000,500000000,97.750,14:00:08,SEN,0,0,0
7,1724565,CO21D,-0.0385,16681.875,0,16039.1000,0,16054.500,-1,,B,16681.875,3,0,16054.500,0,16016.000,SENEBI,276604704,1724565,16039.100,14:00:08,SEN,0,0,0
8,344544,CO26,0.0110,12046.250,0,12209.8027,0,12100.000,-1,,B,12046.250,3,0,12322.312,0,12100.000,SENEBI,41811845,344544,12179.000,14:00:08,SEN,0,0,0
9,27154490,DICP,-0.0068,3161.850,0,3145.1874,0,3180.000,-1,,B,3161.850,3,0,3190.000,0,3120.000,SENEBI,857300235,27154490,3140.500,14:02:27,SEN,0,0,0


## SENEBI - Letras

In [ ]:
data = '{"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## Plazos de negociacion para listar. 
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/senebi-letras', headers=headers, data=data)
senebi_letras = json.loads(response.text)
df_senebi_letras = pd.DataFrame(senebi_letras['data'])

df_senebi_letras

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,tradeHour,securityType,closingPrice,settlementPrice,quantityBid
0,2475000,S28F2,0.0030,92.400,0,92.7580,0,92.912,2,,L,92.400,3,0,92.912,0,92.450,SENEBI,2293854,2475000,92.681,14:00:11,SEN,0,0,0
1,2422216630,S31D1,-0.0057,98.150,0,98.0085,0,98.200,-1,,L,98.150,3,0,98.250,0,97.584,SENEBI,2376317013,2422216630,97.594,14:00:10,SEN,0,0,0
2,514644695,S31E2,0.0022,95.300,0,95.5204,0,95.580,-1,,L,95.300,3,0,95.580,0,95.500,SENEBI,491712399,514644695,95.512,14:00:11,SEN,0,0,0
3,77277082,SD1C,0.0300,0.466,0,0.4800,0,0.480,-1,,L,0.466,3,0,0.480,0,0.480,SENEBI,370930,77277082,0.480,14:00:10,SEN,0,0,0
4,18426930,SD1D,0.0060,0.502,0,0.5050,0,0.505,-1,,L,0.502,3,0,0.505,0,0.505,SENEBI,93056,18426930,0.505,14:00:10,SEN,0,0,0
5,106383000,SM2C,0.0071,0.420,0,0.4230,0,0.423,-1,,L,0.420,3,0,0.423,0,0.423,SENEBI,450000,106383000,0.423,14:00:11,SEN,0,0,0
6,3964756,X23Y2,0.0012,126.113,0,126.3633,0,126.570,2,,L,126.113,3,0,126.570,0,125.950,SENEBI,5005900,3964756,126.260,14:00:10,SEN,0,0,0
7,55446454,X28F2,-0.0021,136.287,0,136.0000,0,136.000,-1,,L,136.287,3,0,136.000,0,136.000,SENEBI,75407177,55446454,136.000,14:00:10,SEN,0,0,0
8,4323144,X29L2,0.0006,115.000,0,115.2600,0,115.640,2,,L,115.000,3,0,115.640,0,114.500,SENEBI,4974641,4323144,115.070,14:00:11,SEN,0,0,0
9,4173692,X30J2,0.0038,119.000,0,119.5500,0,119.750,2,,L,119.000,3,0,119.750,0,119.150,SENEBI,4985475,4173692,119.450,14:00:10,SEN,0,0,0


## SENEBI - ONs

In [ ]:
data = '{"T2":true,"T1":false,"T0":false,"Content-Type":"application/json"}' ## Plazos de negociacion para listar. 
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/senebi-obligaciones-negociables', headers=headers, data=data)
senebi_ons = json.loads(response.text)
df_senebi_ons = pd.DataFrame(senebi_ons['data'])

df_senebi_ons

,tradeVolume,symbol,imbalance,previousSettlementPrice,offerPrice,vwap,numberOfOrders,openingPrice,tickDirection,securityDesc,securitySubType,previousClosingPrice,settlementType,quantityOffer,tradingHighPrice,bidPrice,tradingLowPrice,market,volumeAmount,volume,trade,tradeHour,securityType,closingPrice,settlementPrice,quantityBid
0,3245000,AER1O,0.0224,15390.400,0,15735.6347,0,15760.630,-1,,O,15390.400,3,0,15760.630,0,15636.000,SENEBI,510621907,3245000,15735.652,14:00:09,SEN,0,0,0
1,92673000,BACBO,-0.0129,98.384,0,97.1155,0,97.116,2,,O,98.384,3,0,97.116,0,97.115,SENEBI,90000001,92673000,97.115,14:00:08,SEN,0,0,0
2,604000,CSDOD,0.0075,100.000,0,101.0849,0,101.000,-1,,O,100.000,3,0,102.515,0,100.750,SENEBI,611717,604000,100.750,14:00:09,SEN,0,0,0
3,50000,GNCXD,0.0145,106.161,0,107.7000,0,107.700,-1,,O,106.161,3,0,107.700,0,107.700,SENEBI,53850,50000,107.700,14:00:11,SEN,0,0,0
4,1500000,MRBCC,0.0000,0.000,0,51.5365,0,51.653,0,,O,0.000,3,0,51.653,0,51.450,SENEBI,773223,1500000,51.653,14:00:11,SEN,0,0,0
5,1500000,MRCBO,0.0000,0.000,0,10756.1946,0,10782.560,2,,O,0.000,3,0,10782.560,0,10740.190,SENEBI,161410456,1500000,10740.190,14:00:11,SEN,0,0,0
6,30000,MTCGD,0.0015,112.500,0,112.6700,0,112.670,-1,,O,112.500,3,0,112.670,0,112.670,SENEBI,33801,30000,112.670,14:00:11,SEN,0,0,0
7,250000,RCC9O,-0.0537,21877.217,0,20703.0000,0,20703.000,-1,,O,21877.217,3,0,20703.000,0,20703.000,SENEBI,51757500,250000,20703.000,14:00:08,SEN,0,0,0
8,60000,RPC2D,0.0124,104.500,0,105.8000,0,105.800,-1,,O,104.500,3,0,105.800,0,105.800,SENEBI,63480,60000,105.800,14:00:08,SEN,0,0,0
9,60000,TLC5D,-0.0016,109.872,0,109.7000,0,109.700,-1,,O,109.872,3,0,109.700,0,109.700,SENEBI,65820,60000,109.700,14:00:09,SEN,0,0,0


# Noticias y Eventos

## Listado de hechos relevantes

In [ ]:
data = '{"filter":true,"publishDate":null,"publishToDate":null,"dateEntryFrom":null,"dateEntryTo":null,"Content-Type":"application/json"}' ## Parametros de los filtros.
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/relevant-facts', headers=headers, data=data)
hechos_relevantes = json.loads(response.text)
df_hechos_relevantes = pd.DataFrame(hechos_relevantes['data'])

df_hechos_relevantes

,especie,fecha,tipoArchivo,descarga,referencia,emisor
0,,2021-12-14 10:44:54.0,pdf,398937,Nómina de autoridades designadas en la asamble...,RED SURCOS S.A.
1,BCOM,2021-12-14 10:44:06.0,pdf,398936,Aviso de pago de servicios o renta de Cedear -...,BANCO COMAFI S.A.
2,BCOM,2021-12-14 10:44:06.0,pdf,398935,Aviso de pago de servicios o renta de Cedear -...,BANCO COMAFI S.A.
3,PCRI,2021-12-14 10:42:45.0,pdf,398934,Aviso de pago de Obligaciones Negociables - CL...,PETROQUIMICA COMODORO RIVADAVIA S.A.
4,BCOM,2021-12-14 10:25:44.0,pdf,398926,Aviso de pago de servicios o renta de Cedear -...,BANCO COMAFI S.A.
...,...,...,...,...,...,...
135,FIAT,2021-12-09 10:09:49.0,pdf,398715,Aviso de pago de Obligaciones Negociables - CL...,FCA Compañía Financiera S.A.
136,TECO,2021-12-09 10:08:47.0,pdf,398714,Aviso de Resultado de Colocación - Obligacione...,TELECOM ARGENTINA S. A.
137,PLAZ,2021-12-09 10:05:45.0,pdf,398713,Aviso de pago de Obligaciones Negociables - CL...,Plaza Logística S.R.L.
138,BPA,2021-12-09 10:03:29.0,pdf,398705,Aviso de pago de Fideicomiso Financiero - MERC...,BANCO PATAGONIA S.A.


## Avisos BYMA

In [ ]:
data = '{"filter":true,"fromDate":null,"toDate":null,"Content-Type":"application/json"}' ## Parametros de los filtros.
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/byma-ads', headers=headers, data=data)
avisos_byma = json.loads(response.text)
df_avisos = pd.DataFrame(avisos_byma['data'])

df_avisos

,simbolo,fecha,tipoArchivo,descarga,referencia,emisor
0,ABTC,2021-12-13 17:29:00.0,pdf,398886,Cancelación de listado de Fideicomisos Financi...,TMF Trust Company (ARGENTINA) S.A.
1,ABTC,2021-12-13 14:08:00.0,pdf,398857,Autorización de Listado - Fideicomiso Financie...,TMF Trust Company (ARGENTINA) S.A.
2,PRIO,2021-12-13 12:24:00.0,pdf,398851,Autorización de listado – Letras de Tesorería,Provincia de Río Negro
3,,2021-12-13 11:31:00.0,pdf,398841,Autorización de listado – ON,Albanesi Energía S.A.
4,BCOM,2021-12-10 11:20:00.0,pdf,398786,Certificados de Depósitos Argentinos (CEDEARS)...,BANCO COMAFI S.A.
5,TECO,2021-12-10 10:08:00.0,pdf,398781,Autorización de listado – ON,TELECOM ARGENTINA S. A.
6,TECO,2021-12-10 10:08:00.0,pdf,398781,Autorización de listado – ON,TELECOM ARGENTINA S. A.
7,BVAL,2021-12-09 16:32:00.0,pdf,398751,Autorización de ingreso al régimen de Listado ...,BANCO DE VALORES S.A.
8,BPA,2021-12-09 15:36:00.0,pdf,398744,Autorización de Listado - Fideicomiso Financie...,BANCO PATAGONIA S.A.
9,,2021-12-09 15:24:00.0,pdf,398742,Autorización de Listado de Cuotapartes - Dracm...,Dracma S.A.


## Boletines y Prospectos

In [ ]:
data = '{"date":"13/12/21","Content-Type":"application/json"}' ## Fecha de publicacion
response = s.post('https://open.bymadata.com.ar/vanoms-be-core/rest/api/bymadata/free/bnown/newsletters', headers=headers, data=data)
boletines = json.loads(response.text)
df_boletines = pd.DataFrame(boletines['data'])

df_boletines

,descargaBoletin,I_BOLETIN_ID,descripcionBoletin,tipoArchivoBoletin,tipoArchivoProspecto,descripcionInstrumento,descargaProspecto,emisor
0,398893,6732,Boletín BCBA 13/12/2021,pdf,pdf,Fideicomiso Financiero,398894,BANCO DE VALORES S.A.
1,398893,6732,Boletín BCBA 13/12/2021,pdf,pdf,Obligaciones Negociables,398895,MSU ENERGY S.A.
2,398893,6732,Boletín BCBA 13/12/2021,pdf,pdf,Obligaciones Negociables,398896,CREDITO DIRECTO S.A.
